## Profiling PyTorch training job using Amazon SageMaker Debugger

In this notebook we will demonstrate the deep profiling capability of Amazon SageMaker Debugger

1. Setup
2. Train a PyTorch Tensorflow model for weather prediction with debugger enabled. 
3. Analyze and visualize the system and framework metrics generated by the profiler.
4. Analyze the profiler report generated by SageMaker Debugger.
5. Review and implement recommendations from the profiler report.



### Step 1 - Setup

#### 1.1 Import libraries

In [3]:
import sagemaker
import boto3
from sagemaker import image_uris
from sagemaker.session import Session
from sagemaker.inputs import TrainingInput

from sagemaker.pytorch import PyTorch
from sagemaker.debugger import ProfilerConfig, FrameworkProfile

#### 1.2 Define variables

In [4]:
#Set the s3_bucket to the correct bucket name created in your datascience environment
s3_bucket = 'datascience-environment-notebookinstance--06dc7a0224df'
s3_prefix = 'prepared'
region = boto3.Session().region_name

#### 1.3 Setup service clients

In [5]:
sagemaker_client = boto3.client("sagemaker")
s3_client = boto3.client('s3', region_name=region)

#### 1.4 Training and validation data inputs to training job

In [6]:
##Get the file name at index from the 'prefix' folder
def get_file_in_bucket(prefix,index):
    response = s3_client.list_objects(
        Bucket=s3_bucket,
        Prefix=s3_prefix + "/" + prefix
    )
    ## At '0' index you will find the SUCCESS/FAILURE of file uploades to S3. First data file is at index 1
    file_name = response['Contents'][index]['Key']
    print("Returing file name : " + file_name)
    return file_name

In [7]:
content_type = "csv"

# Define the data type and paths to the training and validation datasets

#Since we are using powerful CPU/GPU instances for training over hours, you can choose to use a single file 
#for training and validation instead of the entrie dataset to save some time and trainging costs.  Change the variable
#use_full_data to True to use the complete dataset
use_full_data=False

#Different train and validation inputs
#define the data type and paths to the training and validation datasets
if use_full_data == False:
    ##Update the csv file names to match the contents in your S3 bucket
    #train_input = TrainingInput("s3://{}/{}/{}/part-00000-2554f113-947e-46bd-be31-9cd75cb4661c-c000.csv".format(s3_bucket, s3_prefix, 'train'), content_type=content_type)
    #validation_input = TrainingInput("s3://{}/{}/{}/part-00000-85addac2-a753-4bc2-b157-26ff8f5d5952-c000.csv".format(s3_bucket, s3_prefix, 'validation'), content_type=content_type)
    train_input = TrainingInput("s3://{}/{}".format(s3_bucket, get_file_in_bucket('train',1)), content_type=content_type)
    validation_input = TrainingInput("s3://{}/{}".format(s3_bucket, get_file_in_bucket('validation',1)), content_type=content_type)
else:
    train_input = TrainingInput("s3://{}/{}/{}/".format(s3_bucket, s3_prefix, 'train'), content_type=content_type, distribution='ShardedByS3Key')
    validation_input = TrainingInput("s3://{}/{}/{}/".format(s3_bucket, s3_prefix, 'validation'), content_type=content_type, distribution='ShardedByS3Key')

Returing file name : prepared/train/part-00000-2554f113-947e-46bd-be31-9cd75cb4661c-c000.csv
Returing file name : prepared/validation/part-00000-85addac2-a753-4bc2-b157-26ff8f5d5952-c000.csv


### Step 2 - Train a PyTorch Tensorflow model for weather prediction with debugger enabled.

#### 2.1 Define training related variables

In [8]:
train_instance_type = "ml.p3.2xlarge"
instance_count = 2

#### 2.2 Define profiler configuration

With the following profiler_config parameter configuration, Debugger calls the default settings of monitoring, collecting system metrics every 500 milliseconds. For collecting framework metrics, you can set target steps and target time intervals in detail.

In [9]:
profiler_config = ProfilerConfig(
    framework_profile_params=FrameworkProfile(start_step=2, num_steps=7)
)

With this profiler_config settings, Debugger will collect system metrics every 500 milliseconds and framework metrics on the specified steps (from step 2 to 9). For a complete list of parameters and profiling configurations, see Configure Debugger Using Amazon SageMaker Python SDK in the Amazon SageMaker Debugger developer guide.

#### 2.3 Create PyTorch Estimator

In [10]:
hyperparameters = {
    "nproc_per_node": 4,
    "nnodes": 2,
}

In [11]:

pt_estimator = PyTorch(
    entry_point="train_pytorch.py",
    source_dir="code",
    role=sagemaker.get_execution_role(),
    instance_count=instance_count,
    instance_type=train_instance_type,
    framework_version="1.6",
    py_version="py3",
    volume_size=1024,
    hyperparameters=hyperparameters,
    #Debugger-specific parameters
    profiler_config=profiler_config,
)

In [ ]:
## Kickoff training
pt_estimator.fit({'train': train_input, 'test': validation_input})

2021-08-09 03:35:03 Starting - Starting the training job...
2021-08-09 03:35:26 Starting - Launching requested ML instancesProfilerReport-1628480102: InProgress
...
2021-08-09 03:35:58 Starting - Preparing the instances for training...............
2021-08-09 03:38:27 Downloading - Downloading input data
2021-08-09 03:38:27 Training - Downloading the training image.......bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-08-09 03:39:33,041 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-08-09 03:39:33,065 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-08-09 03:39:38,777 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-08-09 03:39:38,800 sagemaker_pytorch_container.trainin

### Step 3 - Analyze and visualize the system and framework metrics generated by the profiler.

#### 3.1 Manual Analysis

In [13]:
from smdebug.profiler.system_metrics_reader import S3SystemMetricsReader
import time
#All collected metrics are persisted in S3.  Define path to the profiler artifacts
path = pt_estimator.latest_job_profiler_artifacts_path()
#Create a reader for the system metrics
system_metrics_reader = S3SystemMetricsReader(path)

In [14]:
training_job_name = pt_estimator.latest_training_job.name
print(f"Training job name: {training_job_name}")

Training job name: pytorch-training-2021-08-09-03-35-01-831


In [15]:
training_job_status = ""
training_job_secondary_status = ""
##Wait till the profiler data is available
while system_metrics_reader.get_timestamp_of_latest_available_file() == 0:
    system_metrics_reader.refresh_event_file_list()
    client = sagemaker_client.describe_training_job(TrainingJobName=training_job_name)
    if "TrainingJobStatus" in client:
        training_job_status = f"TrainingJobStatus: {client['TrainingJobStatus']}"
    if "SecondaryStatus" in client:
        training_job_secondary_status = f"TrainingJobSecondaryStatus: {client['SecondaryStatus']}"

    print(
        f"Profiler data from system not available yet. {training_job_status}. {training_job_secondary_status}."
    )
    time.sleep(20)

print("\n\nProfiler data from system is available")



Profiler data from system is available


In [16]:
from datetime import datetime

def timestamp_to_utc(timestamp):
    utc_dt = datetime.utcfromtimestamp(timestamp)
    return utc_dt.strftime("%Y-%m-%d %H:%M:%S")

Now that the data is available we can query and inspect it. We get the latest available timestamp and query all the events within the given time range:

In [17]:
system_metrics_reader.refresh_event_file_list()
#Get the latest event
last_timestamp = system_metrics_reader.get_timestamp_of_latest_available_file()
events = system_metrics_reader.get_events(0, last_timestamp * 1000000)  # UTC time in micro seconds

print(
    "Found",
    len(events),
    "recorded system metric events. Latest recorded event:",
    timestamp_to_utc(last_timestamp / 1000000),
)  # UTC time in seconds to datetime

[2021-08-09 04:27:11.737 ip-172-16-11-227:19699 INFO metrics_reader_base.py:134] Getting 42 event files
Found 126056 recorded system metric events. Latest recorded event: 2021-08-09 03:58:00


We can iterate over the list of recorded events. Let's have a look on the first event.

In [18]:
#Show the first system metric event collected
print(
    "Event name:",
    events[0].name,
    "\nTimestamp:",
    timestamp_to_utc(events[0].timestamp),
    "\nValue:",
    events[0].value,
)

Event name: gpu0 
Timestamp: 2021-08-09 03:38:01 
Value: 0.0


#### 3.2 GPU and CPU usage
MetricHistogram computes a histogram on GPU and CPU utilization values. Bins are between 0 and 100. Good system utilization means that the center of the distribution should be between 80 to 90. In case of multi-GPU training: if distributions of GPU utilization values are not similar it indicates an issue with workload distribution.

The following cell will plot the histograms per metric. In order to only plot specific metrics, define the list select_dimensions and select_events. A dimension can be CPUUtilization, GPUUtilization, GPUMemoryUtilization IOPS. With CPUUtilization dimension, CPU uiltization histogram for each single core and total CPU usage will be plotted. In case of GPU, it will visualize utilization and memory for each GPU. In case of IOPS, it will plot IO wait time per CPU. If select_events is specified then only metrics that match the name in select_metrics will be shown. If neither select_dimensions nor select_events are specified, all available metrics will be visualized. One can also specify a start and endtime.

In [19]:
from smdebug.profiler.analysis.notebook_utils.metrics_histogram import MetricsHistogram

system_metrics_reader.refresh_event_file_list()
metrics_histogram = MetricsHistogram(system_metrics_reader)
metrics_histogram.plot()

[2021-08-09 04:27:23.718 ip-172-16-11-227:19699 INFO metrics_reader_base.py:134] Getting 42 event files
Found 124288 system metrics events from timestamp_in_us:0 to timestamp_in_us:1628481480000000
select events:['.*']
select dimensions:['.*']
filtered_events:{'ReceiveBytesPerSecond', 'cpu3', 'IOPS', 'cpu4', 'cpu5', 'WriteThroughputInBytesPerSecond', 'MemoryUsedPercent', 'cpu1', 'TransmitBytesPerSecond', 'gpu0', 'cpu7', 'total', 'ReadThroughputInBytesPerSecond', 'cpu6', 'cpu0', 'cpu2'}
filtered_dimensions:{'GPUUtilization-nodeid:algo-2', 'CPUUtilization-nodeid:algo-2', 'Platform-nodeid:algo-2', 'Algorithm-nodeid:algo-1', 'GPUUtilization-nodeid:algo-1', 'GPUMemoryUtilization-nodeid:algo-1', 'GPUMemoryUtilization-nodeid:algo-2', '-nodeid:algo-2', 'Algorithm-nodeid:algo-2', 'Platform-nodeid:algo-1', 'CPUUtilization-nodeid:algo-1', 'I/OWaitPercentage-nodeid:algo-1', '-nodeid:algo-1', 'I/OWaitPercentage-nodeid:algo-2'}


filtered_dimensions:{'GPUUtilization-nodeid:algo-2', 'CPUUtilization-nodeid:algo-2', 'Platform-nodeid:algo-2', 'Algorithm-nodeid:algo-1', 'GPUUtilization-nodeid:algo-1', 'GPUMemoryUtilization-nodeid:algo-1', 'GPUMemoryUtilization-nodeid:algo-2', '-nodeid:algo-2', 'Algorithm-nodeid:algo-2', 'Platform-nodeid:algo-1', 'CPUUtilization-nodeid:algo-1', 'I/OWaitPercentage-nodeid:algo-1', '-nodeid:algo-1', 'I/OWaitPercentage-nodeid:algo-2'}


#### 3.3 Read profiling data: framework annotations

In [ ]:
from smdebug.profiler.algorithm_metrics_reader import S3AlgorithmMetricsReader

framework_metrics_reader = S3AlgorithmMetricsReader(path)

events = []
while framework_metrics_reader.get_timestamp_of_latest_available_file() == 0 or len(events) == 0:
    framework_metrics_reader.refresh_event_file_list()
    last_timestamp = framework_metrics_reader.get_timestamp_of_latest_available_file()
    events = framework_metrics_reader.get_events(0, last_timestamp)

    print("Profiler data from framework not available yet")
    time.sleep(20)

print("\n\n Profiler data from framework is available")

[2021-08-09 04:27:29.444 ip-172-16-11-227:19699 INFO algorithm_metrics_reader.py:192] S3AlgorithmMetricsReader created with bucket:sagemaker-us-west-2-802439482869 and prefix:pytorch-training-2021-08-09-03-35-01-831/profiler-output/framework/
[2021-08-09 04:27:29.665 ip-172-16-11-227:19699 INFO metrics_reader_base.py:134] Getting 40 event files
[2021-08-09 04:27:47.400 ip-172-16-11-227:19699 INFO trace_event_file_parser.py:197] Start time for events in uSeconds = 1628480410657345
[2021-08-09 04:27:48.277 ip-172-16-11-227:19699 INFO trace_event_file_parser.py:197] Start time for events in uSeconds = 1628480411445444
[2021-08-09 04:27:49.047 ip-172-16-11-227:19699 INFO trace_event_file_parser.py:197] Start time for events in uSeconds = 1628480410657345
[2021-08-09 04:27:50.022 ip-172-16-11-227:19699 INFO trace_event_file_parser.py:197] Start time for events in uSeconds = 1628480411445444
[2021-08-09 04:27:51.025 ip-172-16-11-227:19699 INFO trace_event_file_parser.py:197] Start time for e

The following code cell retrieves all recorded events from Amazon S3.

In [ ]:
framework_metrics_reader.refresh_event_file_list()
last_timestamp = framework_metrics_reader.get_timestamp_of_latest_available_file()
events = framework_metrics_reader.get_events(0, last_timestamp)

print(
    "Found",
    len(events),
    "recorded framework annotations. Latest event recorded ",
    timestamp_to_utc(last_timestamp / 1000000),
)

Like before we can inspect the recorded events. Since we are reading framework metrics there is now a start and end time for each event.

In [ ]:
print(
    "Event name:",
    events[0].event_name,
    "\nStart time:",
    timestamp_to_utc(events[0].start_time / 1000000000),
    "\nEnd time:",
    timestamp_to_utc(events[0].end_time / 1000000000),
    "\nDuration:",
    events[0].duration,
    "nanosecond",
)

#### 3.4 Outliers in step duration

StepHistogram creates histograms of step duration values. Significant outliers are indication of system bottlenecks. In contrast to SetpTimelineChart it helps identify clusters of step duration values. As a simple example: time spent during training phase (forward and backward pass) will likely be different to time spent during validation phase (forward pass), so we would expect at least two clusters.

In [ ]:
from smdebug.profiler.analysis.notebook_utils.step_histogram import StepHistogram

framework_metrics_reader.refresh_event_file_list()
step_histogram = StepHistogram(framework_metrics_reader)
step_histogram.plot()

#### 3.5 Heatmap
The following code cell creates a heatmap where each row corresponds to one metric (CPU core and GPU utilizations) and x-axis is the duration of the training job. It allows you to more easily spot CPU bottlenecks (utilization on GPU is low but a utilization of one or more cores is high).

In [ ]:
from smdebug.profiler.analysis.notebook_utils.heatmap import Heatmap

view_heatmap = Heatmap(
    system_metrics_reader,
    framework_metrics_reader,
    select_dimensions=["CPU", "GPU"],  # optional - comment this line out to see all dimensions.
    # select_events=["total"],                 # optional - comment this line out to see all events.
    plot_height=900,
)

### Section 4 - Analyze the profiler report generated by SageMaker Debugger.<a id='profiler-report'></a>

In this section we will analyze the report generated by debugger.  We will showcase a few sections of the report.  For complete details, please download the report from the S3 bucket and review.

Also note that the exact details in the report generated for your training job may be different from what you see in this section.  

#### 4.1 View the location of the report generated.

In [ ]:
rule_output_path = pt_estimator.output_path + pt_estimator.latest_training_job.job_name + "/rule-output"
print(
    f"You will find the profiler report under `{rule_output_path}/` after the training has finished"
)

To check if the report is generated, list directories and files recursively

In [ ]:
! aws s3 ls {rule_output_path} --recursive

#### Download the report and rule output files recursively using `aws s3 cp` 
The following command saves all of the rule output files to the **ProfilerReport-1234567890** folder under your current working directory. 

In [ ]:
! aws s3 cp {rule_output_path} ./ --recursive

The following script automatically finds the **ProfilerReport** folder name and returns a link to the downloaded report.

In [ ]:
from IPython.display import FileLink

profiler_report_name = [
    rule["RuleConfigurationName"]
    for rule in pt_estimator.latest_training_job.rule_job_summary()
    if "Profiler" in rule["RuleConfigurationName"]
][0]
profiler_report_name
display(
    "Click link below to view the profiler report",
    FileLink(profiler_report_name + "/profiler-output/profiler-report.html"),
)

For more information about how to find, download, and browse Debugger profiling reports, see SageMaker Debugger Profiling Report in the Amazon SageMaker Debugger developer guide.

### Section 5 - Analyze and Implement recommendations from the report<a id='analyze-profiler-recommendations'></a>

The **Rules Summary** section of the report aggregates all of the rule evaluation results, analysis, rule descriptions, and suggestions. The following table shows a summary of the executed profiler rules. The table is sorted by the rules that triggered most frequently. In training job this was the case for rule LowGPUUtilization. 
It has processed 1001 datapoints and triggered 8 times.

You may see a different rule summary based on the data and the training configuration you use.


<IMG src = 'images/RulesSummary.png'/>

From the analysis so far and the top recommendations from the table above, there seems to be scope for improving resource utilization and make our training efficient.  Based on this change the training configuration settings and re run the training.

#### 5.1 Implement recommendations from the report

Next, we will rerun the training job with the changed configuration. We will keep the same training cluster of 2 p3.ml.2xlarge and update the batch_size to 1024 increasing it from the default value of 64

After second training job with the new settings is complete, there are new system metrics, framework metrics and a new report generated. 

In [ ]:
##Change the batch size to 512 from the default 64
hyperparameters = {
    "batch_size": 1024,
    "nproc_per_node": 4,
    "nnodes": 2,
}

In [ ]:
pt_estimator_updated_batch_size = PyTorch(
    entry_point="train_pytorch.py",
    source_dir="code",
    role=sagemaker.get_execution_role(),
    instance_count=instance_count,
    instance_type=train_instance_type,
    framework_version="1.6",
    py_version="py3",
    volume_size=1024,
    hyperparameters=hyperparameters,
    # Debugger-specific parameters
    profiler_config=profiler_config,
)

In [ ]:
#Kickoff training
pt_estimator_updated_batch_size.fit({'train': train_input, 'test': validation_input})

#### 5.3 Compare the training jobs

To understand the impact of the training configuration changes, compare the report analysis from the two training jobs.  Repeat the process of analyzing the profiler report, implementing the recommendations and comparing with the previous run, till you are satisfied.


In [ ]:
rule_output_path = (
    pt_estimator_updated_batch_size.output_path + pt_estimator_updated_batch_size.latest_training_job.job_name + "/rule-output"
)
print(
    f"You will find the profiler report under {rule_output_path}/ after the training has finished"
)

#### Download the new report and files recursively using `aws s3 cp`

In [ ]:
! aws s3 cp {rule_output_path} ./ --recursive

Retrieve a file link to the new profiling report.

In [ ]:
from IPython.display import FileLink

profiler_report_name = [
    rule["RuleConfigurationName"]
    for rule in pt_estimator_updated_batch_size.latest_training_job.rule_job_summary()
    if "Profiler" in rule["RuleConfigurationName"]
][0]
profiler_report_name
display(
    "Click link below to view the profiler report",
    FileLink(profiler_report_name + "/profiler-output/profiler-report.html"),
)

You should see that the training time has decreased for the training job executed with the updated configuration.  But there is still scope for improvement.  For the next round of triaining, lets keep the updated batch size and change the training instance to a less powerful instance "ml.p2.8xlarge"

#### Next round of training

In [ ]:
train_instance_type = "ml.p2.8xlarge"

pt_estimator_updated_batch_size_instance_size = PyTorch(
    entry_point="train_pytorch.py",
    source_dir="code",
    role=sagemaker.get_execution_role(),
    instance_count=instance_count,
    instance_type=train_instance_type,
    framework_version="1.6",
    py_version="py3",
    volume_size=1024,
    hyperparameters=hyperparameters,
    # Debugger-specific parameters
    profiler_config=profiler_config,
)

In [ ]:
pt_estimator_updated_batch_size_instance_size.fit({'train': train_input, 'test': validation_input})

Now, repeat the steps of retreiving the profiler report and analyzing it.  This time you might notice that the training tie increased slightly but because we used a p2 instance instead of p3, your overall training cost will be reduced. 

## Conclusion

Profiling feature of Amazon SageMaker Debugger is a powerful tool to gain visibility into machine learning training jobs. This notebook provided insight into training resource utilization to identify bottlenecks, analysis of various phases of training and identifying expensive framework functions. The notebook also demonstrated how to analyze and implement profiler recommendations.